1. Explique la diferencia entre descenso de gradiente, descenso de gradiente por mini batches y descenso de
gradiente estocástico. Asegúrese de mencionar las ventajas y desventajas de cada enfoque.

Antes de entrar a detalle entre los distintos tipos de descenso de gradiente, hay que definir el concepto del descenso de gradiente. Este es un algoritmo de optimización iterativo que se utiliza para encontrar el punto mínimo de la función de costo.

La pendiente o gradiente es la derivada de primer orden de la función en el punto actual.

### Descenso de Gradiente:
Este utiliza todo el conjunto de datos en cada iteración para calcular el gradiente de la función de costo.
$$\frac{∂Costo}{∂w} = \frac{1}{N}\sum_{i=1}^{N}  \frac{∂Costo^{(i)}}{∂w}$$

* Ventajas:
    * El gráfico resultante es bastante uniforme debido a que se utiliza el promedio 

* Desventajas:
    * Es muy costoso en datasets grandes
    * No escala bien con datos masivos
    * Se puede quedar atascado en mínimos locales en problemas no convexos

### Descenso de Gradiente Estocástico
Se utiliza una muestra aleatoria en cada iteración.
$$\frac{∂Costo}{∂w} = \frac{∂Costo^{(i)}}{∂w}\text{, para una muestra } i \text{ aleatoria}$$ 

* Ventajas:
    * Cuando se trabaja con datasets más grandes, el costo disminuyirá.
* Desventajas:
    * Este es sensible al escalamiento de características.

### Descenso de Gradiente por mini batches:
Este utiliza únicamente un subconkjunto de k muestras en cada paso. Se utiliza un conjunto de una cantidad fija de muestras de entrenamiento que es menor que el conjunto de datos real.
$$\frac{∂Costo}{∂w} = \frac{1}{k}\sum_{i=1}^{k}  \frac{∂Costo^{(i)}}{∂w}$$

* Ventajas:
    * Aprovecha la vectorización para cálculos paralelos
    * Tiene un balance entre estabilidad y eficiencia
* Desventajas:
    * Aún puede necesitar ajuste de la tasa de aprendizaje

2. Compare y contraste técnicas de extracción de features (feature extraction) y selección de features (feature
selection) en machine learning. De ejemplos de escenarios donde cada técnica sería más apropiada.

### Feature Extraction
Se transforman las features originales en nuevas features, disminuyendo la dimensión pero siempre manteniendo información relevante.
##### Técnicas:
* PCA (Principle Componemts Analysis):
    * Se utilizan los datos originales como input y encontrar una combinación de las features que brinde un resumen de todos los datos. 
* ICA (Independent Component Analysis)
##### Ventajas:
* El dataset es menos complejo
* Requiere menos espacio y menos tiempo de computación
* Hay menos chances de causar ```overfitting``` en el modelo
##### Desventajas:
* El dataset es menos complejo
* Requiere menos espacio y menos tiempo de computación
* Hay menos chances de causar ```overfitting``` en el modelo


### Feature Selection
Como dice el nombre, se trata de seleccionar las columnas relevantes para el estudio que se quiere llegar a hacer. 
##### Técnicas:
* a
* a
##### Ventajas:
* El dataset es menos complejo
* Requiere menos espacio y menos tiempo de computación
* Hay menos chances de causar ```overfitting``` en el modelo
##### Desventajas:
* El dataset es menos complejo
* Requiere menos espacio y menos tiempo de computación
* Hay menos chances de causar ```overfitting``` en el modelo

3. Describa la arquitectura y el funcionamiento de un perceptrón de una sola capa (un tipo de red neuronal sin
backpropagation). Explique cómo aprende y la forma en la que actualiza sus parámetros.